#  3_Enrico_Pre-Processing_Draft.ipynb

This notebook logically follows the merge step and performs data cleaning & preprocessing on merged_stock_income.csv from 2_Enrico_Merged_Draft.ipynb.

✔ What it does:
- Loads the merged dataset from the previous step.

Cleans and prepares the data:
- Drops the "Dividend" column (mostly null).
Converts "Date" to datetime.
- Handles nulls in "Shares Outstanding" intelligently using group-wise .last() and .map().
- Drops or fills missing values using thresholds, medians (for numeric), and modes (for categorical).
- Applies linear interpolation for remaining NaNs.
- Saves the cleaned version (in commented cell).

In [ ]:
import pandas as pd

# Save merged dataset
df_merged = pd.read_csv("/Users/enricotajanlangit/Desktop/merged_stock_income.csv") # - ADAPT TO YOUR LOCAL ENVIRONMENT

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18995999 entries, 0 to 18995998
Data columns (total 36 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Dividend                                  float64
 10  Shares Outstanding                        float64
 11  Currency                                  object 
 12  Fiscal Year                               int64  
 13  Fiscal Period                             object 
 14  

In [3]:
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
0,A,2019-04-15,45846,81.0,81.13,79.91,80.4,77.22,1627268,NaN,...,-22000000.0,-38000000.0,9.190000e+08,NaN,9.190000e+08,152000000.0,1071000000,NaN,1071000000,1071000000
1,A,2019-04-15,45846,81.0,81.13,79.91,80.4,77.22,1627268,NaN,...,-4000000.0,-70000000.0,8.420000e+08,NaN,8.420000e+08,-123000000.0,719000000,NaN,719000000,719000000
2,A,2019-04-15,45846,81.0,81.13,79.91,80.4,77.22,1627268,NaN,...,13000000.0,-79000000.0,1.360000e+09,NaN,1.360000e+09,-150000000.0,1210000000,NaN,1210000000,1210000000
3,A,2019-04-15,45846,81.0,81.13,79.91,80.4,77.22,1627268,NaN,...,-114000000.0,-75000000.0,1.504000e+09,NaN,1.504000e+09,-250000000.0,1254000000,NaN,1254000000,1254000000
4,A,2019-04-15,45846,81.0,81.13,79.91,80.4,77.22,1627268,NaN,...,-11000000.0,-44000000.0,1.339000e+09,NaN,1.339000e+09,-99000000.0,1240000000,NaN,1240000000,1240000000


In [5]:
df_merged.columns

Index(['Ticker', 'Date', 'SimFinId', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding', 'Currency',
       'Fiscal Year', 'Fiscal Period', 'Publish Date', 'Restated Date',
       'Shares (Basic)', 'Shares (Diluted)', 'Revenue', 'Cost of Revenue',
       'Gross Profit', 'Operating Expenses',
       'Selling, General & Administrative', 'Research & Development',
       'Depreciation & Amortization', 'Operating Income (Loss)',
       'Non-Operating Income (Loss)', 'Interest Expense, Net',
       'Pretax Income (Loss), Adj.', 'Abnormal Gains (Losses)',
       'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income',
       'Net Income (Common)'],
      dtype='object')

In [6]:
# example DF for CRWD ticker only 
df_crwd = df_merged[df_merged["Ticker"] == "CRWD"]

# Display the filtered DataFrame
df_crwd.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
4297154,CRWD,2019-06-12,1039026,63.5,67.0,56.0,58.0,58.0,19449162,NaN,...,6283000.0,-442000.0,-139782000.0,NaN,-139782000.0,-1997000.0,-141779000,NaN,-141779000,-141779000
4297155,CRWD,2019-06-12,1039026,63.5,67.0,56.0,58.0,58.0,19449162,NaN,...,4660000.0,3409000.0,-87869000.0,NaN,-87869000.0,-4760000.0,-92629000,NaN,-92629000,-92629000
4297156,CRWD,2019-06-12,1039026,63.5,67.0,56.0,58.0,58.0,19449162,NaN,...,-17475000.0,-21443000.0,-160023000.0,NaN,-160023000.0,-72355000.0,-232378000,NaN,-234802000,-234802000
4297157,CRWD,2019-06-12,1039026,63.5,67.0,56.0,58.0,58.0,19449162,NaN,...,30229000.0,27176000.0,-159883000.0,NaN,-159883000.0,-22402000.0,-182285000,NaN,-183245000,-183245000
4297158,CRWD,2019-06-12,1039026,63.5,67.0,56.0,58.0,58.0,19449162,NaN,...,124812000.0,123174000.0,122817000.0,NaN,122817000.0,-32232000.0,90585000,NaN,89327000,89327000


In [7]:
df_merged.describe()

,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding,Fiscal Year,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
count,1.899600e+07,1.899600e+07,1.899600e+07,1.899600e+07,1.899600e+07,1.899600e+07,1.899600e+07,106554.000000,1.831402e+07,1.899600e+07,...,1.877450e+07,1.667210e+07,1.899313e+07,1.226919e+07,1.899352e+07,1.552953e+07,1.899600e+07,1.872766e+06,1.899600e+07,1.899600e+07
mean,4.209057e+06,7.484808e+02,7.729890e+02,7.156847e+02,7.408929e+02,7.389101e+02,1.972607e+06,0.451973,2.785803e+08,2.020939e+03,...,-1.384801e+08,-2.218519e+08,5.389939e+08,-2.693053e+07,4.405176e+08,-1.204216e+08,4.259542e+08,1.138575e+08,4.243375e+08,4.208690e+08
std,5.109715e+06,5.372795e+04,5.552322e+04,5.085431e+04,5.291014e+04,5.291016e+04,3.043340e+07,1.252695,4.835946e+09,1.381633e+00,...,9.954461e+09,1.260788e+10,2.554847e+10,4.453758e+09,2.736859e+10,8.085709e+09,1.979443e+10,9.199185e+08,1.987330e+10,1.987285e+10
min,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,2.019000e+03,...,-1.007982e+12,-1.381290e+12,-1.574245e+12,-3.176900e+10,-1.574245e+12,-5.054560e+11,-1.394512e+12,-4.306000e+09,-1.409383e+12,-1.409383e+12
25%,4.220400e+05,8.950000e+00,9.200000e+00,8.680000e+00,8.940000e+00,8.480000e+00,7.511250e+04,0.130000,2.158600e+07,2.020000e+03,...,-5.197100e+07,-7.384400e+07,-2.794600e+07,-4.280600e+07,-3.860200e+07,-5.879500e+07,-3.862700e+07,-8.467000e+06,-3.812100e+07,-3.935450e+07
50%,9.952480e+05,2.430000e+01,2.484000e+01,2.374000e+01,2.427000e+01,2.300000e+01,3.426010e+05,0.260000,5.329650e+07,2.021000e+03,...,-4.818000e+06,-1.082600e+07,1.139500e+07,-4.664000e+06,6.014000e+06,-5.553000e+06,4.758000e+06,2.000000e+03,4.771000e+06,4.055000e+06
75%,9.075984e+06,6.344000e+01,6.457000e+01,6.227000e+01,6.340000e+01,6.033000e+01,1.145300e+06,0.500000,1.410862e+08,2.022000e+03,...,2.250000e+05,-2.439290e+05,2.379910e+08,6.840000e+05,2.129520e+08,-2.247500e+04,1.756310e+08,2.400000e+07,1.723650e+08,1.706000e+08
max,1.848257e+07,1.632000e+07,1.632000e+07,1.542400e+07,1.568000e+07,1.568000e+07,1.848998e+10,68.060000,4.752666e+11,2.023000e+03,...,4.258460e+11,2.017040e+11,2.710835e+12,2.809990e+11,2.710835e+12,7.366010e+11,2.205379e+12,2.182700e+10,2.201565e+12,2.201565e+12


In [8]:
df_merged.isna().mean() # 99% of the dividend column is null for all tickers 

Ticker                                      0.000000
Date                                        0.000000
SimFinId                                    0.000000
Open                                        0.000000
High                                        0.000000
Low                                         0.000000
Close                                       0.000000
Adj. Close                                  0.000000
Volume                                      0.000000
Dividend                                    0.994391
Shares Outstanding                          0.035901
Currency                                    0.000000
Fiscal Year                                 0.000000
Fiscal Period                               0.000000
Publish Date                                0.000000
Restated Date                               0.000000
Shares (Basic)                              0.005773
Shares (Diluted)                            0.011968
Revenue                                     0.

# Data Cleaning #

✅ Converted 'Date' into a Datetime Object

✅ Remove duplicate rows if needed

✅ Handled missing values


✅ Drop dividend column (with 99% of nulls)

✅ Shares outstanding -  Replace Nulls with the Latest Value Reported per Ticker 

✅ Shares outstanding -  Replace remaining nulls (for those tickers who have never reported a shares outstanding value) with 0 to preserve data integrity 

✅ Drop Columns with Too Many Missing Values (>50%)

✅ Fill Numerical Columns with Mean/Median imputation (e.g. 'Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses')

✅ Fill Categorical Columns with Mode ('Currency', 'Ticker', 'Fiscal Period')

✅ Drop Rows with Too Many Nulls
If some rows still have excessive missing values, drop them.

✅ Interpolation for Time-Series Data

❌ Outliers - not needed for cleaning. 

Changing Date into a Datetime Object

In [9]:
df_merged['Date'] = pd.to_datetime(df_merged['Date']) # converting the "Date" column to a Datetime Object 

Checking for Duplicates

In [10]:
df_merged.duplicated().sum()  # Count duplicates


0

Dropping the "Divident" column with 99% of nulls 

In [12]:
df_merged = df_merged.drop(columns=["Dividend"])

Handling the null values in "Shares Outstanding" 

In [15]:
# Sort the DataFrame by Ticker and Date in ascending order
df_merged = df_merged.sort_values(by=["Ticker", "Date"])

# Get the latest (most recent) non-null "Shares Outstanding" per Ticker
latest_shares_outstanding = df_merged.groupby("Ticker")["Shares Outstanding"].last()

# Fill missing values using the latest reported value per Ticker
df_merged["Shares Outstanding"] = df_merged["Shares Outstanding"].fillna(df_merged["Ticker"].map(latest_shares_outstanding))

In [16]:
df_merged.isna().mean()

Ticker                                      0.000000
Date                                        0.000000
SimFinId                                    0.000000
Open                                        0.000000
High                                        0.000000
Low                                         0.000000
Close                                       0.000000
Adj. Close                                  0.000000
Volume                                      0.000000
Shares Outstanding                          0.002579
Currency                                    0.000000
Fiscal Year                                 0.000000
Fiscal Period                               0.000000
Publish Date                                0.000000
Restated Date                               0.000000
Shares (Basic)                              0.005773
Shares (Diluted)                            0.011968
Revenue                                     0.083639
Cost of Revenue                             0.

The reason some “Shares Outstanding” values are still null, even after replacing them with the latest available value, is that .last() in groupby("Ticker") selects only the most recent non-null value per Ticker. However, if a Ticker never had a reported non-null value, .last() returns NaN instead of a valid number. As a result, when we try to map and fill missing values, there is no valid value available for replacement, leaving some entries still null.

In [17]:
null_so =  df_merged[df_merged["Shares Outstanding"].isna()]

null_so

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
396145,ADRA,2019-04-15,11817231,32.16,32.16,32.16,32.16,31.43,86,NaN,...,4307581.0,10281.0,3330750.0,-86544.0,3244206.0,NaN,3244206,NaN,3244206,3244206
396146,ADRA,2019-04-15,11817231,32.16,32.16,32.16,32.16,31.43,86,NaN,...,-4056000.0,-4056000.0,47697000.0,-9656000.0,38041000.0,-9423000.0,28618000,NaN,28618000,28618000
396147,ADRA,2019-04-15,11817231,32.16,32.16,32.16,32.16,31.43,86,NaN,...,-11715000.0,-11715000.0,-36308000.0,-8154000.0,-44462000.0,9058000.0,-35404000,NaN,-35404000,-35404000
396148,ADRA,2019-04-16,11817231,32.22,32.22,32.22,32.22,31.48,237,NaN,...,4307581.0,10281.0,3330750.0,-86544.0,3244206.0,NaN,3244206,NaN,3244206,3244206
396149,ADRA,2019-04-16,11817231,32.22,32.22,32.22,32.22,31.48,237,NaN,...,-4056000.0,-4056000.0,47697000.0,-9656000.0,38041000.0,-9423000.0,28618000,NaN,28618000,28618000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18108553,VTOL,2024-03-14,10656416,25.83,26.00,25.53,25.76,25.76,75947,NaN,...,-4593000.0,-41360000.0,28154000.0,-32573000.0,-4419000.0,-11294000.0,-15713000,NaN,-15791000,-15791000
18108554,VTOL,2024-03-14,10656416,25.83,26.00,25.53,25.76,25.76,75947,NaN,...,-35574000.0,-32771000.0,19101000.0,-1089000.0,18012000.0,-24932000.0,-6920000,NaN,-6780000,-6780000
18108555,VTOL,2024-03-15,10656416,25.70,26.28,25.65,25.75,25.75,280498,NaN,...,-22045000.0,-49966000.0,42707000.0,-99347000.0,-56640000.0,355000.0,-56285000,NaN,-56094000,-56094000
18108556,VTOL,2024-03-15,10656416,25.70,26.28,25.65,25.75,25.75,280498,NaN,...,-4593000.0,-41360000.0,28154000.0,-32573000.0,-4419000.0,-11294000.0,-15713000,NaN,-15791000,-15791000


Replacing the Shares Outstanding Values that have never had a reported non-null vallue with 0. 

In [18]:
df_merged["Shares Outstanding"] = df_merged["Shares Outstanding"].fillna(0) # replacing the remaining "Shares Outstanding" with 0

In [21]:
df_merged.columns

Index(['Ticker', 'Date', 'SimFinId', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Shares Outstanding', 'Currency', 'Fiscal Year',
       'Fiscal Period', 'Publish Date', 'Restated Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expenses', 'Selling, General & Administrative',
       'Research & Development', 'Depreciation & Amortization',
       'Operating Income (Loss)', 'Non-Operating Income (Loss)',
       'Interest Expense, Net', 'Pretax Income (Loss), Adj.',
       'Abnormal Gains (Losses)', 'Pretax Income (Loss)',
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income',
       'Net Income (Common)'],
      dtype='object')

In [20]:
null_means = df_merged.isna().mean()
for column, mean in null_means.items():
    if mean > 0:
        print(f"{column}: {mean:.6f}")


Shares (Basic): 0.005773
Shares (Diluted): 0.011968
Revenue: 0.083639
Cost of Revenue: 0.195632
Gross Profit: 0.195577
Operating Expenses: 0.001986
Selling, General & Administrative: 0.054266
Research & Development: 0.539089
Depreciation & Amortization: 0.584403
Operating Income (Loss): 0.000151
Non-Operating Income (Loss): 0.011660
Interest Expense, Net: 0.122336
Pretax Income (Loss), Adj.: 0.000151
Abnormal Gains (Losses): 0.354117
Pretax Income (Loss): 0.000130
Income Tax (Expense) Benefit, Net: 0.182484
Net Extraordinary Gains (Losses): 0.901413


Drop Columns with Too Many Missing Values (>50%)
If a column has too many missing values and isn't critical, it's best to drop it.

columns dropped: Research & Development, Depreciation & Amortization

In [22]:
threshold = 0.5  # 50% missing values threshold
df_merged = df_merged.dropna(axis=1, thresh=len(df_merged) * (1 - threshold))

Fill Numerical Columns with Mean/Median
For numerical data like Revenue, Gross Profit, etc., use median imputation.
Used median imputation instead of mean because median is more robust to outliers

In [24]:
num_cols = ['Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses']
for col in num_cols:
    df_merged[col].fillna(df_merged[col].median(), inplace=True)


/var/folders/59/8bbcmw0s2vlgn_2mn51xcwwc0000gn/T/ipykernel_3523/1711261754.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged[col].fillna(df_merged[col].median(), inplace=True)


Fill Categorical Columns with Mode
For categorical columns like Currency, Ticker, or Fiscal Period.

- Categorical columns like Currency, Ticker, and Fiscal Period contain discrete values.
- The mode is the most frequently occurring category, making it a logical replacement for missing values.
- If we later use one-hot encoding or label encoding, missing values could create problems.
- Filling with the mode ensures that all rows have valid categorical values.

In [25]:
cat_cols = ['Currency', 'Ticker', 'Fiscal Period']
for col in cat_cols:
    df_merged[col].fillna(df_merged[col].mode()[0], inplace=True)


/var/folders/59/8bbcmw0s2vlgn_2mn51xcwwc0000gn/T/ipykernel_3523/1632782131.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged[col].fillna(df_merged[col].mode()[0], inplace=True)


Drop Rows with Too Many Nulls
If some rows still have excessive missing values, drop them.

In [26]:
df_merged.dropna(thresh=len(df_merged.columns) * 0.7, inplace=True)  # Drops rows with >30% missing


Interpolation for Time-Series Data

Uses linear interpolation to estimate missing values between known values.
Works by computing a straight-line equation between two known points and filling missing values accordingly.
- If your dataset follows a time pattern (e.g., financial data), use interpolation.
- For time-series financial data (e.g., stock prices, revenue), missing values often occur due to holidays, reporting delays, or system gaps.
- Interpolation helps smooth data trends rather than using arbitrary imputation like mean/median.
- It ensures that missing values follow the trend instead of creating unnatural spikes.

In [28]:
df_merged = df_merged.sort_values('Date')  # Ensure sorting before interpolation
df_merged.interpolate(method='linear', inplace=True)


/var/folders/59/8bbcmw0s2vlgn_2mn51xcwwc0000gn/T/ipykernel_3523/2969921151.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_merged.interpolate(method='linear', inplace=True)


In [ ]:
df_merged.isna().mean()

Ticker                                      0.000000e+00
Date                                        0.000000e+00
SimFinId                                    0.000000e+00
Open                                        0.000000e+00
High                                        0.000000e+00
Low                                         0.000000e+00
Close                                       0.000000e+00
Adj. Close                                  0.000000e+00
Volume                                      0.000000e+00
Shares Outstanding                          0.000000e+00
Currency                                    0.000000e+00
Fiscal Year                                 0.000000e+00
Fiscal Period                               0.000000e+00
Publish Date                                0.000000e+00
Restated Date                               0.000000e+00
Shares (Basic)                              0.000000e+00
Shares (Diluted)                            0.000000e+00
Revenue                        

In [ ]:
# Define the file path
#file_path = "/Users/enricotajanlangit/Desktop/pre-processed.csv"

# Save the DataFrame as a CSV file
#df_merged.to_csv(file_path, index=False)

#print(f"File saved successfully at: {file_path}")

OSError: [Errno 28] No space left on device